# Kagle inclass https://www.kaggle.com/c/simplesentiment/overview

In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np

from itertools import product
import warnings
from tqdm import tqdm

import re

In [2]:
import nltk
#nltk.download('movie_reviews')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('rslp')

import nltk.stem as st

In [3]:
import spacy

#from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#from gensim.sklearn_api import W2VTransformer
import gensim.downloader as api

unable to import 'smart_open.gcs', disabling that module


In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score 

#from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [5]:
import xgboost as xgb
from lightgbm import LGBMClassifier

In [6]:
PATH_DATA = os.path.join(Path.cwd(), 'data')
PATH_SUBM = os.path.join(Path.cwd(), 'submissions')

In [31]:
df = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_train.tsv'), 
                    header = None, 
                    index_col = None,
                    sep = '\t',
                   )
df.columns = ['text', 'target']
df.shape

(2000, 2)

In [32]:
df.head()

,text,target
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [33]:
df.target.value_counts()

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1    1274
0     726
Name: target, dtype: int64

Классы явно не сбалансированы.   
Т.к. мы работаем с текстами есть следующие варианты:   
    1. добавить негативные примеры. + увеличение выборки, баланс классов. - необходимо найти размеченную, подходящую под тематику, выборку   
    2. дублировать некоторые строки, например те, в которых ошибается модель   
    3. настроить параметр class_weights
    

### Приводим к строчным (на всякий) и очищаем по порядку:

In [34]:
# - все спецсимволы
# - все цифры
# - заменяем множественные пробелы на единичные

In [35]:
clean_text = lambda x: re.sub(r"\s+", ' ', 
                              re.sub(r"[\d+]", '',
                                     re.sub(r"[^\w\s]", '', x.lower()).strip()
                                    )
                             )

In [36]:
df['text_cl'] = df.text.map(clean_text)

In [37]:
#tagged_data[:5]

In [38]:
#lem_text("don't"), lem_text("i'll")

In [39]:
#stem_text("don't"), stem_text("i'll")

### Лемматизация & стемминг

In [40]:
lemm = st.WordNetLemmatizer()
lem_text = lambda x: ' '.join([lemm.lemmatize(el) for el in x.split()])


stemm = st.RSLPStemmer()
stem_text = lambda x: ' '.join([stemm.stem(el) for el in x.split()])

In [41]:
df['text_cl'] = df.text.map(clean_text)
#df['text_cl'] = df.text_cl.map(lem_text)
#df['text_cl'] = df.text_cl.map(stem_text)

In [42]:
df['text_cl'] = df.text_cl.map(lambda x: x.split())

In [43]:
df.head()

,text,target,text_cl
0,"2 . take around 10,000 640x480 pictures .",1,"[take, around, x, pictures]"
1,i downloaded a trial version of computer assoc...,1,"[i, downloaded, a, trial, version, of, compute..."
2,the wrt54g plus the hga7t is a perfect solutio...,1,"[the, wrtg, plus, the, hgat, is, a, perfect, s..."
3,i dont especially like how music files are uns...,0,"[i, dont, especially, like, how, music, files,..."
4,i was using the cheapie pail ... and it worked...,1,"[i, was, using, the, cheapie, pail, and, it, w..."


Загружаем готовый wordvector

https://github.com/RaRe-Technologies/gensim-data

In [126]:
#word_vectors = api.load("glove-wiki-gigaword-300")
#word_vectors = api.load("glove-twitter-200")#, return_path = True
word_vectors = api.load("word2vec-google-news-300")

C:\Users\_v010ch_\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:1498: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\_v010ch_/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz'>
  limit=limit, datatype=datatype)


In [127]:
word_vectors

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'C:\\Users\\****/gensim-data\\glove-wiki-gigaword-300\\glove-wiki-gigaword-300.gz'

### Подготавливаем к обучению по сетке

In [107]:
train, test, train_target, test_target = train_test_split(df.text_cl, df.target, 
                                                          test_size = 0.2, 
                                                          stratify = df.target, 
                                                          random_state = 52138,
                                                         )

### Параметры для перебора

In [108]:
pipe_cnt = Pipeline([#('w2v', W2VTransformer(size=100, min_count=1, seed=1)),
                     ('clf', LogisticRegression()),
                      ])

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [109]:
parameters_v0 = {
        'clf': [#LogisticRegression(), 
                LinearSVC(),
                #SGDClassifier(), 
                #RandomForestClassifier(),
                #xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0),
                #xgb.XGBClassifier(eval_metric = 'auc'),
               ],
    }

In [116]:
parameters = [
            {'clf': [LogisticRegression(max_iter = 150)]}, 
            {'clf': [LinearSVC(max_iter = 1500)]},
            {'clf': [SGDClassifier()]}, 
            {'clf': [RandomForestClassifier()]},
            {'clf': [xgb.XGBClassifier(eval_metric = 'auc', use_label_encoder=False)]},
                     #xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0),
            {'clf': [LGBMClassifier()]},
]

In [117]:
# Трансфорование данных (отзыва) в вектор
# требуется исключить слова, которые не встречались, т.к. выдаст ошибку
# и полученные вектораотзыва усредняем по столбцам
def transform_x2v(inp_model, inp_series):
    transformed_ndarray = np.ndarray((inp_series.shape[0], inp_model.gensim_model.vector_size))
    
    for row, idx in enumerate(inp_series.index):
        
        tr_string = ''
        for wrd in inp_series[idx]:
            if wrd in inp_model.gensim_model.wv.vocab:
                tr_string = ' '.join([tr_string, wrd])

        transformed_ndarray[row] = np.mean(inp_model.transform(tr_string.split()), axis = 0)
    
    return transformed_ndarray

In [118]:
# Трансфорование данных (отзыва) в вектор
# требуется исключить слова, которые не встречались, т.к. выдаст ошибку
# и полученные вектораотзыва усредняем по столбцам
def transform_x2v_kv(inp_wv, inp_series):
    transformed_ndarray = np.ndarray((inp_series.shape[0], inp_wv.vector_size))
    
    for row, idx in enumerate(inp_series.index):
        vect = []    
        for wrd in inp_series[idx]:
            if wrd in inp_wv:
                vect.append(inp_wv.get_vector(wrd))

        transformed_ndarray[row] = np.mean(vect, axis = 0)
    
    return transformed_ndarray

In [131]:
%%time
#warnings.filterwarnings("ignore")

train_x2v = np.nan_to_num(transform_x2v_kv(word_vectors, train))

grid = GridSearchCV(pipe_cnt, parameters, cv = 5, scoring = 'roc_auc', verbose = 1, n_jobs=-1)
ret = grid.fit(train_x2v, train_target)


#warnings.filterwarnings("always")
#warnings.filterwarnings("default")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Wall time: 13.4 s


Посмотрим на несколько лучших результатов

In [132]:
grid.best_score_, grid.best_estimator_['clf'].__class__

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0.8636084672595828, sklearn.svm._classes.LinearSVC)

In [133]:
[(el0, el1) for el0, el1 in zip(grid.cv_results_['mean_test_score'], grid.cv_results_['params'])]

[(0.8625631242771201, {'clf': LogisticRegression(max_iter=150)}),
 (0.8636084672595828, {'clf': LinearSVC(max_iter=1500)}),
 (0.8592270108750839, {'clf': SGDClassifier()}),
 (0.8326044476145895, {'clf': RandomForestClassifier()}),
 (0.8438609086428558,
  {'clf': XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
                 colsample_bynode=None, colsample_bytree=None, eval_metric='auc',
                 gamma=None, gpu_id=None, importance_type='gain',
                 interaction_constraints=None, learning_rate=None,
                 max_delta_step=None, max_depth=None, min_child_weight=None,
                 missing=nan, monotone_constraints=None, n_estimators=100,
                 n_jobs=None, num_parallel_tree=None, random_state=None,
                 reg_alpha=None, reg_lambda=None, scale_pos_weight=None,
                 subsample=None, tree_method=None, use_label_encoder=False,
                 validate_parameters=None, verbosity=None)}),
 (0.8457348235492

### Попробуем определить причины ошибок по тесту

In [139]:
#train_x2v = np.nan_to_num(transform_x2v(model_x2v, train))

model = LinearSVC(max_iter = 2500)
model.fit(train_x2v, train_target)

LinearSVC(max_iter=2500)

In [142]:
test_pred = model.predict(np.nan_to_num(transform_x2v_kv(word_vectors, test)))

In [143]:
confusion_matrix(test_target, test_pred)

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[ 91,  54],
       [ 39, 216]], dtype=int64)

только половина отрицательных отзывов, посмотрим их

In [144]:
for idx in range(test.shape[0]):
    if test_pred[idx] != test_target[test.index[idx]]:
        print(test.index[idx],test_target[test.index[idx]], df.loc[test.index[idx]].text)

1585 0 although i find it more convenient to use 1-touch dialing , this phone does not have voice dialing . 
1736 0 it is slightly larger than an ipod , no doubt about it . 
1650 0 the gprs web access is sloooooooooooooow !
617 1 no skips , and it hasn 't faltered once ( despite your inevitable drop or two .
506 1 however , the battery life is good enough you should n't be overly concerned with turning this phone off . 
931 1 it is the most bang-for-the-buck out there . 
315 1 norton pops up a window about the intrusions and gives me as much detail as possible about where the intrusion is coming from .
207 1 so far it has been very durable ( other than the little pads on the earbuds ) , but those aren 't really necessary as i find them comfortable on their own .
966 0 recenly the variable speed control went out .
1972 1 my first impression was positive . 
1777 0 it also has a stock tracking app built in ( also useless ) and a somewhat useful application which converts all types of metr

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Если посмотреть на отзывы, в которых модель ошиблась - то в них нет ничего необычного.   
Большая часть однозначно определяется как положительные или отрицательные.   
Предполагаю, что влияют 2 фактора: несбалансированность класса и не больша выборка для обучения
Попробуем исправить, настроив class_weights

# Пробуем настроить параметры алгоритма

#### в 3-4 итерации. в начале ищем приближенные лучшие значения(закомментированные параметры ниже),   
#### потом их уточняем

In [145]:
pipe_cnt = Pipeline([('clf', SVC()),
                      ])

In [168]:
model_parameters = {
    'clf__max_iter' : [2500, 2000],# 1500],
    'clf__kernel': ['rbf', 'linear'], #'poly', 'sigmoid',],
    #'clf__C': np.logspace(-1, 2, 20),
    'clf__C': np.linspace(3.5, 4, 10),
    'clf__class_weight': [{0: 0.53 - shift, 1:0.47 + shift} for shift in np.linspace(-0.01, 0.01, 20)] + ['balanced'],
    #'clf__class_weight': [{0: 0.56 - shift, 1:0.44 + shift} for shift in np.linspace(-0.01, 0.01, 20)]# + ['balanced'],
}

In [169]:
%%time

train_x2v = np.nan_to_num(transform_x2v_kv(word_vectors, train))

grid_tune = GridSearchCV(pipe_cnt, model_parameters, cv = 5, scoring = 'roc_auc', verbose = 1, n_jobs=-1)
grid_tune.fit(train_x2v, train_target)
grid_tune.best_estimator_

Fitting 5 folds for each of 840 candidates, totalling 4200 fits
Wall time: 14min 59s


Pipeline(steps=[('clf',
                 SVC(C=3.6666666666666665,
                     class_weight={0: 0.531578947368421, 1: 0.4684210526315789},
                     max_iter=2500))])

In [170]:
grid_tune.best_params_

C:\Users\_v010ch_\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'clf__C': 3.6666666666666665,
 'clf__class_weight': {0: 0.531578947368421, 1: 0.4684210526315789},
 'clf__kernel': 'rbf',
 'clf__max_iter': 2500}

In [171]:
grid_tune.best_score_

0.8729543895365397

0.8729543895365397
{'clf__C': 3.6666666666666665,
 'clf__class_weight': {0: 0.531578947368421, 1: 0.4684210526315789},
 'clf__kernel': 'rbf',
 'clf__max_iter': 2500}

0.8729120383785698
{'clf__C': 3.7368421052631575,
 'clf__class_weight': {0: 0.5342105263157895, 1: 0.46578947368421053},
 'clf__kernel': 'rbf',
 'clf__max_iter': 2500}
 
0.8729205209935433
('clf', SVC(C=3.79269019073225,
class_weight={0: 0.5368421052631579, : 0.4631578947368421},
max_iter=2500))

Посмотрим на несколько лучших результатов, после чего уточняем параметры и повторяем (несколько раз) GridSearchCV

In [173]:
interested_tune = np.where(grid_tune.cv_results_['rank_test_score'] <= 5)
for field in model_parameters.keys():
    print(field)
    for idx in interested_tune:
        print(grid_tune.cv_results_['param_' + field][idx])

clf__max_iter
[2500 2000 2500 2000 2500 2000]
clf__kernel
['rbf' 'rbf' 'rbf' 'rbf' 'rbf' 'rbf']
clf__C
[3.611111111111111 3.611111111111111 3.6666666666666665 3.6666666666666665
 3.7777777777777777 3.7777777777777777]
clf__class_weight
[{0: 0.5305263157894737, 1: 0.46947368421052627}
 {0: 0.5305263157894737, 1: 0.46947368421052627}
 {0: 0.531578947368421, 1: 0.4684210526315789}
 {0: 0.531578947368421, 1: 0.4684210526315789}
 {0: 0.5378947368421053, 1: 0.4621052631578947}
 {0: 0.5378947368421053, 1: 0.4621052631578947}]


In [174]:
grid_tune.cv_results_['mean_test_score'][np.where(grid_tune.cv_results_['rank_test_score'] <= 5)]

array([0.8729291 , 0.8729291 , 0.87295439, 0.87295439, 0.8729373 ,
       0.8729373 ])

Остановимся на следующих параметрах:    

word_vectors - "word2vec-google-news-300"   
SVC
kernel = 'rbf'
C =  3.6666666666666665
сlass_weight = {0: 0.531578947368421, 1: 0.4684210526315789}
max_iter = 2500

Получим модель и submission для выбранных параметров

### Подготавливаем данные для обучения финальной модели и прогноза

In [175]:
df_test = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_test.tsv'),
                        index_col = None,
                        sep = '\t',
                     )
df_test.shape

(500, 2)

In [176]:
#df_test.head(), df_test.tail()

In [177]:
subm = pd.read_csv(os.path.join(PATH_DATA, 'products_sentiment_sample_submission.csv'))
subm.shape

(500, 2)

In [178]:
#subm.head(), subm.tail()

In [179]:
test_df = df_test.text.map(clean_text)
#test_df = test_df.map(lem_text)
#test_df = test_df.map(stem_text)
test_df = test_df.map(lambda x: x.split())

In [180]:
test_x2v = np.nan_to_num(transform_x2v_kv(word_vectors, test_df))

### Финальная модель для прогноза и прогноз

In [181]:
pred = grid_tune.best_estimator_.predict(test_x2v)
subm.y = pred

In [183]:
subm.to_csv(os.path.join(PATH_SUBM, 'gensim_w2v_gglnews300_svc_tuned_no_ls.csv'), index = False)

pb 0.80888